In [2]:
pip install bs4

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     ----------------------------------- -- 133.1/143.0 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 143.0/143.0 kB 1.7 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1282 sha256=abe4f2c3641e204a73100c9da0297e231b688b4f30e9558ff1eff937a5e0dccd
  Stored in directory: c:\users\irani\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\Irani\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
pip install yfinance

     ---------------------------------------- 0.0/60.3 kB ? eta -:--:--
     ---------------------------------------- 60.3/60.3 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
     -- ------------------------------------- 0.2/2.6 MB 5.1 MB/s eta 0:00:01
     ----- ---------------------------------- 0.4/2.6 MB 3.8 MB/s eta 0:00:01
     ------- -------------------------------- 0.5/2.6 MB 4.0 MB/s eta 0:00:01
     ---------- ----------------------------- 0.7/2.6 MB 3.6 MB/s eta 0:00:01
     ------------- -------------------------- 0.9/2.6 MB 3.6 MB/s eta 0:00:01
     --------------- ------------------------ 1.0/2.6 MB 3.7 MB/s eta 0:00:01
     ------------------ --------------------- 1.2/2.6 MB 3.8 MB/s eta 0:00:01
     --------------------- ------------------ 1.4/2.6 MB 3.7 MB/s eta 0:00:01
     ------------------------ --------------- 1.6/2.6 MB 3.7 MB/s eta 0:00:01
     -------------------------- ------------- 1.7/2.6 MB 3.6 MB/s eta 0:00:


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\Irani\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import requests
from bs4 import BeautifulSoup
import yfinance as yf

# Retrieve stock data from Yahoo Finance API
stock_data = yf.download("IDFCFIRSTB.NS", start="2010-01-01", end="2023-05-22")  # Replace "HDFCBANK.NS" with the HDFC Bank stock symbol

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data['Close'].values.reshape(-1, 1))

# Web scrape news articles from moneycontrol.com
url = 'https://www.moneycontrol.com/news/tags/hdfc-bank.html'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
articles = soup.find_all('div', class_='common_news_main')

# Extract relevant information from scraped articles
news_data = []
for article in articles:
    title = article.find('a').text.strip()
    date = article.find('span', class_='mr10').text.strip()
    content = article.find('p').text.strip()
    news_data.append({'Title': title, 'Date': date, 'Content': content})

# Convert news data to a DataFrame
news_df = pd.DataFrame(news_data)

# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)  # 80% for training, 20% for testing
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Convert the data into sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 10  # Length of the input sequence
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions for the next day
last_sequence = X_test[-1]  # Take the last sequence from the testing set
next_day = model.predict(last_sequence.reshape(1, seq_length, 1))  # Predict the next day's price

# Inverse scale the prediction
next_day = scaler.inverse_transform(next_day)

print("Predicted stock price for the next day:", next_day[0, 0])

[*********************100%***********************]  1 of 1 completed
Epoch 1/10
47/47 [==============================] - 6s 10ms/step - loss: 0.0309
Epoch 2/10
47/47 [==============================] - 1s 13ms/step - loss: 0.0023
Epoch 3/10
47/47 [==============================] - 1s 20ms/step - loss: 0.0018
Epoch 4/10
47/47 [==============================] - 1s 18ms/step - loss: 0.0018
Epoch 5/10
47/47 [==============================] - 1s 16ms/step - loss: 0.0017
Epoch 6/10
47/47 [==============================] - 1s 14ms/step - loss: 0.0018
Epoch 7/10
47/47 [==============================] - 1s 18ms/step - loss: 0.0017
Epoch 8/10
47/47 [==============================] - 1s 21ms/step - loss: 0.0016
Epoch 9/10
47/47 [==============================] - 1s 16ms/step - loss: 0.0016
Epoch 10/10
1/1 [==============================] - 1s 1s/step
Predicted stock price for the next day: 65.95639


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- IDFCFIRSTB.NS: ValueError('unconverted data remains:  14:00:00')


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime

# Prompt user for stock symbol and prediction time
stock_symbol = input("Enter the stock symbol (e.g., AAPL, HDFCBANK.NS): ")
prediction_date = input("Enter the prediction date (ddmmyyyy): ")
prediction_time = input("Enter the prediction time (hh:mm AM/PM): ")

# Convert prediction date and time to datetime object
prediction_datetime = datetime.strptime(prediction_date + prediction_time, "%d%m%Y%I:%M %p")

# Retrieve stock data from Yahoo Finance API
stock_data = yf.download(stock_symbol, start="2010-01-01", end=prediction_datetime)

# Preprocess the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data['Close'].values.reshape(-1, 1))

# Web scrape news articles from moneycontrol.com
url = f"https://www.moneycontrol.com/news/tags/{stock_symbol.lower()}.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
articles = soup.find_all('div', class_='common_news_main')

# Extract relevant information from scraped articles
news_data = []
for article in articles:
    title = article.find('a').text.strip()
    date = article.find('span', class_='mr10').text.strip()
    content = article.find('p').text.strip()
    news_data.append({'Title': title, 'Date': date, 'Content': content})

# Convert news data to a DataFrame
news_df = pd.DataFrame(news_data)

# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)  # 80% for training, 20% for testing
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Convert the data into sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 50  # Length of the input sequence
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(seq_length, 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Find the corresponding index in the stock data
prediction_index = stock_data.index.get_loc(prediction_datetime, method='nearest')

# Prepare the input data for prediction
last_sequence = scaled_data[prediction_index - seq_length:prediction_index].reshape(1, seq_length, 1)

# Make the prediction
predicted_price = model.predict(last_sequence)
predicted_price = scaler.inverse_transform(predicted_price)

print("Predicted stock price for", stock_symbol, "at", prediction_datetime, ":", predicted_price[0, 0])


[*********************100%***********************]  1 of 1 completed
Epoch 1/10
82/82 [==============================] - 7s 34ms/step - loss: 0.0086
Epoch 2/10
82/82 [==============================] - 3s 34ms/step - loss: 3.1004e-04
Epoch 3/10
82/82 [==============================] - 3s 35ms/step - loss: 2.6561e-04
Epoch 4/10
82/82 [==============================] - 3s 36ms/step - loss: 2.6021e-04
Epoch 5/10
82/82 [==============================] - 3s 36ms/step - loss: 2.5858e-04
Epoch 6/10
82/82 [==============================] - 3s 37ms/step - loss: 2.4039e-04
Epoch 7/10
82/82 [==============================] - 3s 37ms/step - loss: 2.3620e-04
Epoch 8/10
82/82 [==============================] - 3s 38ms/step - loss: 2.5180e-04
Epoch 9/10
82/82 [==============================] - 3s 36ms/step - loss: 2.1424e-04
Epoch 10/10
82/82 [==============================] - 3s 38ms/step - loss: 1.8922e-04


C:\Users\Irani\AppData\Local\Temp\ipykernel_9848\3136603989.py:72: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  prediction_index = stock_data.index.get_loc(prediction_datetime, method='nearest')


1/1 [==============================] - 1s 992ms/step
Predicted stock price for HDFCBANK.NS at 2023-05-24 10:00:00 : 1578.56


import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime

# Prompt user for stock symbols and prediction time
stock_symbols = input("Enter the stock symbols (comma-separated, e.g., AAPL,GOOGL,HDFCBANK.NS): ")
prediction_date = input("Enter the prediction date (ddmmyyyy): ")
prediction_time = input("Enter the prediction time (hh:mm AM/PM): ")

# Convert stock symbols to a list
stock_symbols = [symbol.strip() for symbol in stock_symbols.split(",")]

# Convert prediction date and time to datetime object
prediction_datetime = datetime.strptime(prediction_date + prediction_time, "%d%m%Y%I:%M %p")

# Retrieve stock data from Yahoo Finance API for each stock symbol
stock_data = {}
for symbol in stock_symbols:
    stock_data[symbol] = yf.download(symbol, start="2010-01-01", end=prediction_datetime)

# Preprocess the data for each stock
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = {}
for symbol, data in stock_data.items():
    scaled_data[symbol] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Split the data into training and testing sets for each stock
train_data = {}
test_data = {}
for symbol, data in scaled_data.items():
    train_size = int(len(data) * 0.8)  # 80% for training, 20% for testing
    train_data[symbol] = data[:train_size]
    test_data[symbol] = data[train_size:]

# Convert the data into sequences for each stock
seq_length = 50  # Length of the input sequence
X_train = {}
y_train = {}
X_test = {}
y_test = {}
for symbol in stock_symbols:
    X_train[symbol], y_train[symbol] = create_sequences(train_data[symbol], seq_length)
    X_test[symbol], y_test[symbol] = create_sequences(test_data[symbol], seq_length)

# Train and evaluate the LSTM model for each stock
models = {}
for symbol in stock_symbols:
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=(seq_length, 1)))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train[symbol], y_train[symbol], epochs=10, batch_size=32)
    models[symbol] = model

# Predict the stock prices for each stock
for symbol in stock_symbols:
    last_sequence = X_test[symbol][-1]  # Get the last sequence from the test data
    last_sequence = last_sequence.reshape(1, seq_length, 1)  # Reshape for prediction
    predicted_price = models[symbol].predict(last_sequence)  # Predict the next stock price
    predicted_price = scaler.inverse_transform(predicted_price)  # Scale back the predicted price
    print("Stock Symbol:", symbol)
    print("Predicted Stock Price:", predicted_price[0][0])
    print()